In [2]:
from tensorflow.keras.models import load_model

2024-04-02 14:11:29.977189: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-02 14:11:30.447351: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
from scipy.optimize import minimize

In [3]:
import numpy as np

In [5]:
model = load_model('surogate.h5')

2024-03-27 16:50:02.114880: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-03-27 16:50:02.115353: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [6]:
# 特定频率
def loss_freq(s_para):
    # 能量
    E = np.square(s_para)
    E11 = E[:,0] + E[:,1]
    E21 = E[:,2] + E[:,3]
    E31 = E[:,4] + E[:,5]
    E41 = E[:,6] + E[:,7]

    # 压制
    loss1 = E11 - E21 - E31 + E41
    # 比例
    loss2 = np.abs(E21 / (E31 + E21) - 0.7)
    # phase
    loss3 = np.abs(np.arctan2(s_para[:,3], s_para[:,2]) - np.arctan2(s_para[:,5], s_para[:,4]) - np.pi / 4)
    # 能量守恒
    loss4 = np.abs(np.sum(E, axis=1) - 1)
    
    return loss1 + loss2 + loss3 + loss4

In [7]:
def dgn_obj(s1, s2, s3):
    return np.max([loss_freq(s1), loss_freq(s2), loss_freq(s3)], axis=0)

In [4]:
method = 'Newton-CG'
options = {'disp': True, 'maxiter': 1000}

In [8]:
initial_guess = np.random.uniform(lb, ub, (1, 12))

In [10]:
def objective_function(x):
    y_pred1 = model.predict(np.concatenate((2.4, x), axis=-1), batch_size=NUM_NODES, verbose=0)
    y_pred2 = model.predict(np.concatenate((2.5, x), axis=-1), batch_size=NUM_NODES, verbose=0)
    y_pred3 = model.predict(np.concatenate((2.6, x), axis=-1), batch_size=NUM_NODES, verbose=0)
    return dgn_obj(y_pred1, y_pred2, y_pred3)

In [12]:
result = minimize(objective_function, initial_guess,
                  method=method,
                  jac='2-point',  # 自动估算梯度
                  hess='3-point',  # 自动估算Hessian
                  options=options)

/tmp/ipykernel_17376/1580269334.py:1: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(objective_function, initial_guess,


ValueError: Jacobian is required for Newton-CG method

In [11]:
# 参数数量，迭代次数
NUM_NODES = 1
EPOCHS = 1000

In [9]:
freq1 = np.ones((NUM_NODES, 1), dtype=np.float32) * 2.4
freq2 = np.ones((NUM_NODES, 1), dtype=np.float32) * 2.5
freq3 = np.ones((NUM_NODES, 1), dtype=np.float32) * 2.6

In [5]:
# 物理边界
lb = [1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4]
ub = [5, 5, 5, 5, 5, 5, 5, 5, 100, 100, 100, 100]
bounds = list(zip(lb, ub))

In [17]:
# 定义目标函数
def objective_function(x):
    y_pred1 = model.predict(np.concatenate((freq1, x), axis=-1), batch_size=NUM_NODES, verbose=0)
    y_pred2 = model.predict(np.concatenate((freq2, x), axis=-1), batch_size=NUM_NODES, verbose=0)
    y_pred3 = model.predict(np.concatenate((freq3, x), axis=-1), batch_size=NUM_NODES, verbose=0)
    return dgn_obj(y_pred1, y_pred2, y_pred3)

In [18]:
x0 = np.random.uniform(lb, ub, (NUM_NODES, 12))

In [19]:
# 设置basinhopping参数
kwargs = {
    'niter': EPOCHS,  # 总迭代次数
    'T': 1.0,       # 接受概率的温度参数（模拟退火中的温度）
    'stepsize': 0.5, # 步长参数，决定每次尝试跳跃时改变变量的程度
    'disp': True,
    'minimizer_kwargs': {
        'method': 'L-BFGS-B',
        'bounds': bounds
    },  # 局部最小化方法和其参数
}

In [20]:
# 执行basinhopping算法
result = basinhopping(objective_function, x0, **kwargs)

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (12,)  and requested shape (10000,)

In [14]:
# 输出结果
print('全局最小值点:', result.x)
print('全局最小值:', result.fun)

全局最小值点: [ 3.76499141  5.          4.00936613  2.63361956  1.          2.10577186
  3.15875565  3.2510583  11.6142477  12.25386023 52.8935892  23.42361531]
全局最小值: 3.6786904335021973
